In [16]:
import pandas as pd
import gseapy as gp
from scipy.stats import rankdata
from itertools import combinations
import numpy as np

In [34]:
#File with the ranked list of genes of the expression matrix
#If it is the expression matrix the first column, named GENE, should contain the gene symbols
#Genes with negative expression values or zero will be discarded from analysis
filename = 'example_ranked_gene_list.xls'

#Is it the expression matrix? True or False
exp_mat = True

#Database to perform the GSEA
db_name = '../data/MSigDB/hallmarks.gmt'

#Name of the analysis it will be used for output folder and summary file in results/ folder
analysis_name = 'example_ranked'

Note of advanced parameters: 

<p> By default GSEA will generate the enrichment plots of the top-20 enriched pathways if you want to change this number you may change "graph_num"</p>
<p> By default it will do 1,000 random permutations. This means the lowest P-value you will obtain is 10-3. You may lower this number to speed-up the enrichment or increase it to have more statistical power (you may have a memory error here)</p>
<p> If you input a ranked file it is assumed that lower values means downregulation and higher up-regulation </p>

In [35]:
#GSEA function
def run_GSEA_function(genes_rankval,db_name,analysis_name):
    #Run GSEA
    pre_res = gp.prerank(rnk=genes_rankval[['GENE','RANK']],
                             gene_sets=db_name,
                             outdir='../results/'+analysis_name,format='png',
                             graph_num=20,
                             permutation_num=1000,
                             weighted_score_type=0)
    results = pd.DataFrame(pre_res.res2d)
    results = results.reset_index()
    #Dump a summary of the results into an Excel file
    results.to_excel('../results/'+analysis_name+'_GSEAresults_SUMMARY.xls',index=False)

def compute_log2(x):
    if x <= 0:
        return -666
    else:
        return np.log2(float(x))

In [36]:
if exp_mat: 
    #If it is an expression matrix
    exp = pd.read_csv('../data/example_expression_matrix.tsv',sep='\t',index_col=False,header=0)
    print('expression data loaded')
    combi_cols = combinations([column for column in exp.columns.tolist() if column!='GENE'],2)
    for pair in combi_cols:
        print('computing gene FC for',pair)
        exp_c = exp[['GENE',pair[0],pair[1]]].copy(deep=True)
        exp_c[pair[0]+'_log2'] = exp_c[pair[0]].apply(lambda x:compute_log2(x))
        exp_c[pair[1]+'_log2'] = exp_c[pair[1]].apply(lambda x:compute_log2(x))
        exp_c = exp_c[(exp_c[pair[0]+'_log2']!=-666)&(exp_c[pair[1]+'_log2']!=-666)]
        exp_c['FC'] = exp_c[pair[0]+'_log2'] - exp_c[pair[1]+'_log2']
        exp_c['RANK'] = rankdata(exp_c['FC'].tolist())
        print('running GSEA for',pair,' this might take hours')
        run_GSEA_function(exp_c[['GENE','RANK']],db_name,analysis_name+'_'+pair[0]+'_vs_'+pair[1]) #run GSEA
else:
    #It is a ranked list of genes
    genes_rankval = pd.read_excel('../data/'+filename,names=['GENE','FC']) #Load data
    genes_rankval['RANK'] = rankdata(genes_rankval['FC'].tolist()) #Compute rank 
    run_GSEA_function(genes_rankval,db_name,analysis_name) #run GSEA


expression data loaded
computing gene FC for ('TCGA-OR-A5J1', 'TCGA-OR-A5J2')
running GSEA for ('TCGA-OR-A5J1', 'TCGA-OR-A5J2')  this might take hours
computing gene FC for ('TCGA-OR-A5J1', 'TCGA-OR-A5J3')
running GSEA for ('TCGA-OR-A5J1', 'TCGA-OR-A5J3')  this might take hours
computing gene FC for ('TCGA-OR-A5J2', 'TCGA-OR-A5J3')
running GSEA for ('TCGA-OR-A5J2', 'TCGA-OR-A5J3')  this might take hours


Explanation of the columns in the Excel summary file
<ul>
<li>Term: name of the pathway enriched</li>
<li>es: enrichment score</li>
<li>nes: normalized enrichment score (the one to look at!)</li>
<li>pval: P-value of the enrichment</li>
<li>fdr: Corrected(P-value) of the enrichment by FDR (the one to look at!). Recommended threshold: fdr < 0.25</li>
<li>gene_set_size: number of genes in the pathway</li>
<li>matched_size: number of genes in the pathway and in your input file</li>
<li>genes: genes in the pathway and in your input file</li>
</ul>